# John Doty's Global FPE Calibration

by Matthew Wampler-Doty and Ed Bokhour

Remember that whenever you power-cycle the Observatory Simulator, you should set `preload=True` below.

When you are running this notbook and it has *not* been power cycled, you should set `preload=False`.

In [1]:
from tessfpe.dhu.fpe import FPE
from tessfpe.dhu.unit_tests import check_house_keeping_voltages
fpe1 = FPE(1, debug=False, preload=False, FPE_Wrapper_version='6.1.1')
print fpe1.version
if check_house_keeping_voltages(fpe1):
    print "Wrapper load complete. Interface voltages OK."

Observatory Simulator Version 1.5c - Aug 31 2015 10:58:39
Wrapper load complete. Interface voltages OK.


We assume that there is a slight global error in the housekeeping that may be corrected by a linear transformation:

$$ f(x) := m \cdot x + c $$

To calculus $c$, we can average the biases in the housekeeping, but we must first convert them to ADUs from uA by using the `unscale_value` function.

Note that because of statistical variation, we collect 50 samples:

In [2]:
def estimate_c_param(fpe,samples=50):
    from tessfpe.dhu.house_keeping import unscale_value
    from tessfpe.data.housekeeping_channels import housekeeping_channels
    sample_data = []
    for _ in range(samples):
        analogue_house_keeping = fpe.house_keeping["analogue"]
        biases = [unscale_value(analogue_house_keeping[k],
                                16,
                                housekeeping_channels[k]["low"],
                                housekeeping_channels[k]["high"])
                  for k in fpe1.house_keeping["analogue"]
                  if 'bias' in k]
        for b in biases:
            sample_data.append(b)
    
    return sum(sample_data) / len(sample_data)

Below run the above estimation; this should be approximately $-4.83$ based on previous calculations

In [3]:
estimate_c_param(fpe1)

-5.003333333333515

Next we can use the known voltages from the power supply to recover the slope $m$, which is done as follows.  Once again, 50 samples are collected because we are scientists at MIT:

In [4]:
def unscale_1_8_f(x):
    from tessfpe.data.housekeeping_channels import housekeeping_channels
    from tessfpe.dhu.house_keeping import unscale_value
    low = housekeeping_channels['+1.8f']['low']
    high = housekeeping_channels['+1.8f']['high']
    return unscale_value(x, 16, low, high)

def unscale_1_f(x):
    from tessfpe.data.housekeeping_channels import housekeeping_channels
    from tessfpe.dhu.house_keeping import unscale_value
    low = housekeeping_channels['+1f']['low']
    high = housekeeping_channels['+1f']['high']
    return unscale_value(x, 16, low, high)

def estimate_m_param(fpe,samples=50):
    slope_samples = []
    for _ in range(samples):
        a_ = unscale_1_8_f(1.807)
        b_ = unscale_1_f(0.998)
        a = unscale_1_8_f(fpe1.house_keeping['analogue']['+1.8f'])
        b = unscale_1_f(fpe1.house_keeping['analogue']['+1f'])
        slope_samples.append((a_ - b_) / (a - b))
    return sum(slope_samples) / len(slope_samples)

Once again, we run the calculation; this should be $\approx 1$

In [5]:
estimate_m_param(fpe1)

1.000875830298136